In [ ]:
%load_ext autoreload

# Canonical form

In [ ]:
import numpy as np
import scipy.linalg
from mps.state import *
from mps.state import _truncate_vector

## Environment 

A matrix product state form facilitates a decomposition of a complex quantum state in terms of a given site and a left and right environment, formed by the rest of the quantum subsystems.

<img src="figures/environment.svg" style="max-width:90%; width: 22em">

Mathematically, we are performing a Schmidt-type decomposition of the state
$$|\psi\rangle = \sum_{\alpha,i_3,\beta} C^{i_3}_{\alpha\beta}|L^\alpha\rangle|i_3\rangle|R^\beta\rangle$$
with some states $|L^\alpha\rangle$ and $|R^\beta\rangle$ that define a many-body basis for the left and right environments of our central subsystem.

## Canonical form

We claim that the MPS is in *canonical form* with respect to the site $i$ when its left and right tensors define orthonormal basis for their many-body states. In other words, when
$$\langle L^\alpha | L^{\alpha'}\rangle = \delta_{\alpha,\alpha'}$$
$$\langle R^\beta | R^{\beta'}\rangle = \delta_{\beta,\beta'}$$


We can achieve a canonical form by imposing that the tensors to the left and to the right of our subsystem be isometries. In our particular example
$$\sum_i A^{i}_{1,\alpha} A^{i *}_{1,\alpha'} = \delta_{\alpha,\alpha'}$$
$$\sum_{i,\alpha} B^{i}_{\alpha,\beta} B^{i *}_{\alpha,\beta'} = \delta_{\beta,\beta'}$$
$$\sum_{i,\beta} D^{i}_{\alpha,\beta} D^{i *}_{\alpha',\beta} = \delta_{\alpha,\alpha'}$$
$$\sum_i E^{i}_{\alpha,1}E^{i *}_{\alpha',1} = \delta_{\alpha,\alpha'}$$

Or graphically, we can summarize these equations as follows

<img src="figures/canonical-conditions.svg" style="max-width:95%; width:60em">

## Advantages

There are various places where a canonical form becomes very useful. One is when we want to take expectation values of observables. Suppose we wish to compute the average of an observable acting on the third site above
$$\bar{O} = \langle \psi |1 \otimes 1 \otimes O \otimes 1 \otimes 1 |\psi\rangle.$$
If the state is in canonical form, the expectation value can be obtained as a contraction over the local tensors
$$\bar{O} = \sum_{ij\alpha\beta} O_{ij} C^{j*}_{\alpha\beta} C^{i}_{\alpha\beta}$$

Once more, there is a graphical way to express this relation:

<img src="figures/local-expectation-value.svg" style="width: 8em">

The locality of this relation is particularly useful when optimizing expectation values: we can tune the affected tensors independently, until the optimal condition is reached.

## Canonicalizing a tensor

From the images above we guess that there are different canonical conditions depending on whether we come from the left or the right of a given site. If we come from the left, we can start with a tensor that does not satisfy a canonical form and construct a new one that does.

<img src="figures/canonical-split-right.svg" style="max-width: 90%; width: 35em">

Take for instance the tensor $B^{i}_{\alpha\beta},$ which does not satisfy a canonical form. What we do is to reinterpret $B$ as a matrix $C_{x,\beta}$, where the index $x=[\alpha,i_2]$ is born out of joining two legs of the tensor. This matrix admits a singular value decomposition (SVD)
$$C_{x\beta} = \sum_\gamma U_{x,\gamma} s_\gamma V_{\gamma,\beta}$$
with two unitary matrices $U, V$ and a diagonal matrix of non-negative values $s_\gamma.$

We define the tensor
$$\bar{B}^{i_2}_{\alpha\gamma} := U_{x,\gamma}$$
as our new tensor for the second site. The remaining transformations $s V$ are shifted to the next site and, in this particular case, used to update the $C$ tensor to a new tensor
$$\bar{C}_{\gamma\delta}^{i_3} = s_\gamma V_{\gamma\sigma} C_{\sigma\delta}^{i_3}.$$

We can implement this idea as a generic algorithm that updates an MPS, assuming that it is in canonical form up to site $i\pm1$ and moving to site $i$. The algorithm takes the MPS, a generic tensor, the site to update and the direction along which we are moving.

The first part of the algorithm is the splitting of the tensors. We create two functions for this task, `ortho_right()` and `ortho_left()` depending on the direction.

In [ ]:
# file: mps/state.py

def _ortho_right(A, tol, normalize):
    α, i, β = A.shape
    U, s, V = scipy.linalg.svd(np.reshape(A, (α*i, β)), full_matrices=False,
                               lapack_driver='gesvd')
    s, err = _truncate_vector(s, tol, None)
    if normalize:
        s /= np.linalg.norm(s)
    D = s.size
    return np.reshape(U[:,:D], (α, i, D)), np.reshape(s, (D, 1)) * V[:D, :], err


def _ortho_left(A, tol, normalize):
    α, i, β = A.shape
    U, s, V = scipy.linalg.svd(np.reshape(A, (α, i*β)), full_matrices=False,
                               lapack_driver='gesvd')
    s, err = _truncate_vector(s, tol, None)
    if normalize:
        s /= np.linalg.norm(s)
    D = s.size
    return np.reshape(V[:D,:], (D, i, β)), U[:, :D] * np.reshape(s, (1, D)), err

With the functions above we can now construct the actual update of the MPS at a given site.

In [ ]:
# file: mps/state.py


def _update_in_canonical_form(Ψ, A, site, direction, tolerance, normalize):
    """Insert a tensor in canonical form into the MPS Ψ at the given site.
    Update the neighboring sites in the process.
    
    Arguments:
    ----------
    Ψ = MPS in CanonicalMPS form
    A = tensor to be orthonormalized and inserted at "site" of MPS 
    site = the index of the site with respect to which 
    orthonormalization is carried out
    direction = if greater (less) than zero right (left) orthonormalization
    is carried out
    tolerance = truncation tolerance for the singular values 
    (see _truncate_vector in File 1a - MPS class)           
    """
    if direction > 0:
        if site+1 == Ψ.size:
            Ψ[site] = A
            err = 0.
        else:
            Ψ[site], sV, err = _ortho_right(A, tolerance, normalize)
            site += 1
            Ψ[site] = np.einsum('ab,bic->aic', sV, Ψ[site])
    else:
        if site == 0:
            Ψ[site] = A
            err = 0.
        else:
            Ψ[site], Us, err = _ortho_left(A, tolerance, normalize)
            site -= 1
            Ψ[site] = np.einsum('aib,bc->aic', Ψ[site], Us)
    return site, err

This algorithm can be used iteratively to make an MPS into canonical form with respect to a given site, even if it was not previously so.

In [ ]:
# file: mps/state.py


def _canonicalize(Ψ, center, tolerance, normalize):
    err = 0.
    for i in range(0, center):
        center, errk = _update_in_canonical_form(Ψ, Ψ[i], i, +1, tolerance, normalize)
        err += errk
    for i in range(Ψ.size-1, center, -1):
        center, errk = _update_in_canonical_form(Ψ, Ψ[i], i, -1, tolerance, normalize)
        err += errk
    return err

Applying a two-site operator to an MPS yields a composite MPS tensor of two-sites. We use left/right orthonormalization to split this tensor into two one-site tensors using the functions below.

In [ ]:
# file: mps/state.py

def left_orth_2site(AA, tolerance, normalize, dimension):
    α, d1, d2, β = AA.shape
    Ψ = np.reshape(AA, (α*d1, β*d2))
    U, S, V = scipy.linalg.svd(Ψ, full_matrices=False, lapack_driver='gesvd')
    S, err = _truncate_vector(S, tolerance, None)
    if normalize:
        S /= np.linalg.norm(S)
    D = S.size
    U = np.reshape(U[:,:D], (α, d1, D))
    SV = np.reshape( np.reshape(S, (D,1)) * V[:D,:], (D,d2,β) )
    return U, SV, err

def right_orth_2site(AA, tolerance, normalize, dimension):
    α, d1, d2, β = AA.shape
    Ψ = np.reshape(AA, (α*d1, β*d2))
    U, S, V = scipy.linalg.svd(Ψ, full_matrices=False, lapack_driver='gesvd')
    S, err = _truncate_vector(S, tolerance, dimension)
    if normalize:
        S /= np.linalg.norm(S)
    D = S.size    
    US = np.reshape(U[:,:D] * np.reshape(S, (1, D)), (α, d1, D))
    V = np.reshape(V[:D,:], (D,d2,β))
    return US, V, err

## Canonical form MPS

We can use this idea to implement an MPS class that is in canonical form with respect to one site. This site may change as we update the MPS, but it is always kept track of.

In [ ]:
# file: mps/state.py


class CanonicalMPS(MPS):
    """Canonical MPS class.

    This implements a Matrix Product State object with open boundary
    conditions, that is always on canonical form with respect to a given site.
    The tensors have three indices, `A[α,i,β]`, where `α,β` are the internal
    labels and `i` is the physical state of the given site.

    Parameters
    ----------
    data      -- a list of MPS tensors, an MPS or a CanonicalMPS
    center    -- site to make the canonical form. If defaults either to
                 the center of the CanonicalMPS or to zero.
    error     -- norm-2 squared truncation error that we carry on
    tolerance -- truncation tolerance when creating the canonical form
    normalize -- normalize the state after finishing the canonical form
    """

    #
    # This class contains all the matrices and vectors that form
    # a Matrix-Product State.
    #
    def __init__(self, data, error=0, center=None, normalize=False, tolerance=DEFAULT_TOLERANCE):
        super(CanonicalMPS, self).__init__(data, error=error)
        if isinstance(data, CanonicalMPS):
            self.center = data.center
            self._error = data._error
            if center is not None:
                self.recenter(center, tolerance, normalize)
        else:
            self.center = center = self._interpret_center(0 if center is None else center)
            self.update_error(_canonicalize(self, center, tolerance, normalize))
        if normalize:
            A = self[center]
            self[center] = A / np.linalg.norm(A)

    @classmethod
    def fromvector(ψ, dimensions, center=0, normalize=False,
                   tolerance=DEFAULT_TOLERANCE):
        return CanonicalMPS(mps.state.vector2mps(ψ, dimensions, tolerance),
                            center=center, normalize=normalize,
                            tolerance=tolerance)

    def norm2(self):
        """Return the square of the norm-2 of this state, ‖ψ‖^2 = <ψ|ψ>."""
        A = self._data[self.center]
        return np.vdot(A, A)
    
    def left_environment(self, site):
        start = min(site, self.center)
        ρ = expectation.begin_environment(self[start].shape[0])
        for A in self[start:site]:
            ρ = expectation.update_left_environment(A, A, ρ)
        return ρ
    
    def right_environment(self, site):
        start = max(site, self.center)
        ρ = expectation.begin_environment(self[start].shape[-1])
        for A in self[start:site:-1]:
            ρ = expectation.update_right_environment(A, A, ρ)
        return ρ
    
    def expectation1(self, operator, site=None):
        """Return the expectated value of `operator` acting on the given `site`."""
        if site is None or site == self.center:
            A = self._data[self.center]
            return np.vdot(A, np.einsum('ij,ajb->aib', operator, A))
        else:
            return expectation.expectation1(ψ, operator, site)

    def entanglement_entropyAtCenter(self):
        d1, d2, d3 = self._data[self.center].shape
        u,s,v = np.linalg.svd(np.reshape(self._data[self.center], (d1*d2,d3)))
        return -np.sum(2 * s * s * np.log2(s))
    
    def update_canonical(self, A, direction, tolerance=DEFAULT_TOLERANCE, normalize=False):
        self.center, err = _update_in_canonical_form(self, A, self.center,
                                                     direction, tolerance, normalize)
        self.update_error(err)
        return err
        
    def update_2site(self, AA, site, direction, tolerance=DEFAULT_TOLERANCE, normalize=False, dimension=None):
        """Split a two-site tensor into two one-site tensors by 
        left/right orthonormalization and insert the tensor in 
        canonical form into the MPS Ψ at the given site and the site
        on its left/right. Update the neighboring sites in the process.

        Arguments:
        ----------
        Ψ = MPS in CanonicalMPS form
        AA = two-site tensor to be split by orthonormalization
        site = the index of the site with respect to which 
        orthonormalization is carried out
        direction = if greater (less) than zero right (left) orthonormalization
        is carried out
        tolerance = truncation tolerance for the singular values 
        (see _truncate_vector in File 1a - MPS class)           
        """
        assert site <= self.center <= site+1
        if direction < 0:
            self._data[site], self._data[site+1], err = right_orth_2site(AA, tolerance, normalize, dimension)
            self.center = site
        else:
            self._data[site], self._data[site+1], err = left_orth_2site(AA, tolerance, normalize, dimension)
            self.center = site+1
        self.update_error(err)
        return err
               
    def _interpret_center(self, center):
        """Converts `center` into an integer between [0,size-1], with the
        convention that -1 = size-1, -2 = size-2, etc. Trows an exception of
        `center` if out of bounds."""
        size = self.size
        if 0 <= center < size:
            return center
        center += size
        if 0 <= center < size:
            return center
        raise IndexError()

    def recenter(self, center, tolerance=DEFAULT_TOLERANCE, normalize=False):
        """Update destructively the state to be in canonical form with respect
        to a different site."""
        center = self._interpret_center(center)
        old = self.center
        if center != old:
            dr = +1 if center > old else -1
            for i in range(old, center, dr):
                self.update_canonical(self._data[i], dr, tolerance, normalize)
        return self

    def __copy__(self):
        #
        # Return a copy of the MPS with a fresh new array.
        #
        return type(self)(self, self.center)

    def copy(self):
        """Return a fresh new TensorArray that shares the same tensor as its
        sibling, but which can be destructively modified without affecting it.
        """
        return self.__copy__()

----


# Tests

To properly test the canonical forms we have to verify that the tensors are close to isometries. The following function is a helper for that.

In [5]:
# file: mps/test/test_canonical.py
import unittest
from mps.test.tools import *
from mps.state import DEFAULT_TOLERANCE, _update_in_canonical_form, _canonicalize, CanonicalMPS

class TestCanonicalForm(unittest.TestCase):

    def test_local_update_canonical(self):
        #
        # We verify that _update_in_canonical_form() leaves a tensor that
        # is an approximate isometry.
        #
        def ok(Ψ, normalization=False):
            for i in range(Ψ.size-1):
                ξ = Ψ.copy()
                _update_in_canonical_form(ξ, ξ[i], i, +1,
                                          DEFAULT_TOLERANCE,
                                          normalization)
                self.assertTrue(approximateIsometry(ξ[i], +1))
            for i in range(1, Ψ.size):
                ξ = Ψ.copy()
                _update_in_canonical_form(ξ, ξ[i], i, -1,
                                          DEFAULT_TOLERANCE,
                                          normalization)
                self.assertTrue(approximateIsometry(ξ[i], -1))

        test_over_random_mps(ok)
        test_over_random_mps(lambda ψ: ok(ψ, normalization=True))

    def test_canonicalize(self):
        #
        # We verify _canonicalize() transforms an MPS into an equivalent one
        # that is in canonical form and represents the same state, up to
        # a reasonable tolerance.
        #
        def ok(Ψ, normalization=False):
            for center in range(Ψ.size):
                ξ = Ψ.copy()
                _canonicalize(ξ, center, DEFAULT_TOLERANCE, normalization)
                #
                # All sites to the left and to the right are isometries
                #
                for i in range(center):
                    self.assertTrue(approximateIsometry(ξ[i], +1))
                for i in range(center+1, ξ.size):
                    self.assertTrue(approximateIsometry(ξ[i], -1))
                #
                # Both states produce the same wavefunction
                #
                self.assertTrue(similar(ξ.tovector(), Ψ.tovector()))
        test_over_random_mps(ok)

    def test_canonical_mps(self):
        #
        # We verify _canonicalize() transforms an MPS into an equivalent one
        # that is in canonical form and represents the same state, up to
        # a reasonable tolerance.
        #
        def ok(Ψ):
            for center in range(Ψ.size):
                ξ = CanonicalMPS(Ψ, center=center)
                #
                # All sites to the left and to the right are isometries
                #
                for i in range(center):
                    self.assertTrue(approximateIsometry(ξ[i], +1))
                for i in range(center+1, ξ.size):
                    self.assertTrue(approximateIsometry(ξ[i], -1))
                #
                # Both states produce the same wavefunction
                #
                self.assertTrue(similar(ξ.tovector(), Ψ.tovector()))
                #
                # The norm is correct
                #
                self.assertAlmostEqual(ξ.norm2()/Ψ.norm2(), 1.0)
                #
                # Local observables give the same
                #
                O = np.array([[0, 0], [0, 1]])
                nrm2 = ξ.norm2()
                self.assertAlmostEqual(ξ.expectation1(O)/nrm2,
                                       Ψ.expectation1(O, center)/nrm2)
                #
                # The canonical form is the same when we use the
                # corresponding negative indices of 'center'
                #
                χ = CanonicalMPS(Ψ, center=center-Ψ.size)
                for i in range(Ψ.size):
                    self.assertTrue(similar(ξ[i], χ[i]))
        test_over_random_mps(ok)

    def test_environments(self):
        #
        # Verify that the canonical form is indeed canonical and the
        # environment is orthogonal
        #
        def ok(Ψ):
            for center in range(Ψ.size):
                ξ = CanonicalMPS(Ψ, center=center)
                Lenv = super(CanonicalMPS, ξ).left_environment(center)
                Renv = super(CanonicalMPS, ξ).left_environment(center)
                print(Lenv)
                self.assertTrue(almostIdentity(Lenv))
                self.assertTrue(almostIdentity(Renv))
        test_over_random_mps(ok)

    def test_canonical_mps_normalization(self):
        #
        # We verify CanonicalMPS(...,normalize=True) normalizes the
        # vector without really changing it.
        #
        def ok(Ψ):
            for center in range(Ψ.size):
                ξ1 = CanonicalMPS(Ψ, center=center, normalize=False)
                ξ2 = CanonicalMPS(Ψ, center=center, normalize=True)
                self.assertAlmostEqual(ξ2.norm2(), 1.0)
                self.assertTrue(similar(ξ1.tovector()/np.sqrt(ξ1.norm2()),
                                        ξ2.tovector()))
        test_over_random_mps(ok)

    def test_canonical_mps_copy(self):
        #
        # Copying a class does not invoke _canonicalize and does not
        # change the tensors in any way
        #
        def ok(Ψ):
            for center in range(Ψ.size):
                ψ = CanonicalMPS(Ψ, center=center, normalize=True)
                ξ = ψ.copy()
                self.assertEqual(ξ.size, ψ.size)
                self.assertEqual(ξ.center, ψ.center)
                for i in range(ξ.size):
                    self.assertTrue(np.all(np.equal(ξ[i], ψ[i])))
        test_over_random_mps(ok)

In [6]:
suite1 = unittest.TestLoader().loadTestsFromNames(['__main__.TestCanonicalForm'])
unittest.TextTestRunner(verbosity=2).run(suite1);

test_environments (__main__.TestCanonicalForm) ... 

[[1.]]
[[1. 0.]
 [0. 1.]]
[[ 1.00000000e+00  1.14491749e-16  2.77555756e-17 -1.11022302e-16]
 [ 1.14491749e-16  1.00000000e+00 -2.77555756e-17  3.33066907e-16]
 [ 4.16333634e-17  0.00000000e+00  1.00000000e+00 -9.71445147e-17]
 [-1.38777878e-16  2.77555756e-16 -1.38777878e-16  1.00000000e+00]]
[[ 1.00000000e+00  2.40144005e-16  1.24141149e-17  9.60603125e-17
  -2.16840434e-17  2.77555756e-17  2.16840434e-18  1.31838984e-16]
 [ 1.32015167e-16  1.00000000e+00 -1.80221506e-16 -4.27175656e-17
  -3.46944695e-18 -8.67361738e-19 -9.49761103e-17  4.68375339e-17]
 [ 9.59518923e-18 -1.79625195e-16  1.00000000e+00  2.77555756e-17
   1.14491749e-16 -3.15719673e-16 -1.94289029e-16  1.38777878e-16]
 [ 9.97465999e-17 -5.78963960e-17  2.08166817e-17  1.00000000e+00
   4.16333634e-16 -3.46944695e-16 -5.55111512e-17  0.00000000e+00]
 [-2.77555756e-17  0.00000000e+00  1.07552856e-16  3.60822483e-16
   1.00000000e+00 -3.88578059e-16 -3.05311332e-16 -1.66533454e-16]
 [ 4.68375339e-17 -9.10729825e-18 -3.486

ok

----------------------------------------------------------------------
Ran 1 test in 3.557s

OK
